In [101]:
# !pip3 install pandas
# !pip3 install -U scikit-learn
# !pip3 install xgboost

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
submit = pd.read_csv('./data/sample_submission.csv')

In [3]:
train.head()

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,36 months,8 years,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,TRAIN_00004,18000000,60 months,Unknown,RENT,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B


In [4]:
train.columns

Index(['ID', '대출금액', '대출기간', '근로기간', '주택소유상태', '연간소득', '부채_대비_소득_비율', '총계좌수',
       '대출목적', '최근_2년간_연체_횟수', '총상환원금', '총상환이자', '총연체금액', '연체계좌수', '대출등급'],
      dtype='object')

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96294 entries, 0 to 96293
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            96294 non-null  object 
 1   대출금액          96294 non-null  int64  
 2   대출기간          96294 non-null  object 
 3   근로기간          96294 non-null  object 
 4   주택소유상태        96294 non-null  object 
 5   연간소득          96294 non-null  int64  
 6   부채_대비_소득_비율   96294 non-null  float64
 7   총계좌수          96294 non-null  int64  
 8   대출목적          96294 non-null  object 
 9   최근_2년간_연체_횟수  96294 non-null  int64  
 10  총상환원금         96294 non-null  int64  
 11  총상환이자         96294 non-null  float64
 12  총연체금액         96294 non-null  float64
 13  연체계좌수         96294 non-null  float64
 14  대출등급          96294 non-null  object 
dtypes: float64(4), int64(5), object(6)
memory usage: 11.0+ MB


In [6]:

# 상관 관계 보기
train.corr()

,대출금액,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
대출금액,1.000000,0.273975,0.002263,0.212309,-0.014775,0.379061,0.615680,0.011853,0.000182
연간소득,0.273975,1.000000,-0.044594,0.145403,0.040260,0.137552,0.114311,0.000333,0.009854
부채_대비_소득_비율,0.002263,-0.044594,1.000000,0.058830,0.000699,-0.016180,0.018565,0.000711,0.000051
총계좌수,0.212309,0.145403,0.058830,1.000000,0.112092,0.083615,0.116299,0.000037,0.023652
최근_2년간_연체_횟수,-0.014775,0.040260,0.000699,0.112092,1.000000,-0.010048,0.008095,0.004612,0.133632
총상환원금,0.379061,0.137552,-0.016180,0.083615,-0.010048,1.000000,0.412385,0.015379,-0.005190
총상환이자,0.615680,0.114311,0.018565,0.116299,0.008095,0.412385,1.000000,0.057838,0.009257
총연체금액,0.011853,0.000333,0.000711,0.000037,0.004612,0.015379,0.057838,1.000000,0.006620
연체계좌수,0.000182,0.009854,0.000051,0.023652,0.133632,-0.005190,0.009257,0.006620,1.000000


In [106]:
categoric_col = train.select_dtypes(include='object').columns # 카테고리형 컬럼
numeric_col = train.select_dtypes(include='int64').columns # 수치형 컬럼

In [107]:
categoric_col

Index(['ID', '대출기간', '근로기간', '주택소유상태', '대출목적', '대출등급'], dtype='object')

In [108]:
#카테고리화 수치형으로 변환
ordinal_encoder = OrdinalEncoder()
train[categoric_col] = ordinal_encoder.fit_transform(train[categoric_col])

In [109]:
pred_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

In [110]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from xgboost import XGBClassifier

In [118]:
X = train.drop(['대출등급'],axis = 1)
Y = train['대출등급']

In [119]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2 ,random_state=42,stratify=Y)

In [120]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((77035, 14), (19259, 14), (77035,), (19259,))

In [121]:
model = XGBClassifier()

In [122]:
model.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [123]:
y_pred = model.predict(x_test)

In [124]:
f1_score(y_test,y_pred, average="macro")

0.7802786794276975

In [125]:
categoric_col_test = test.select_dtypes(include='object').columns # 카테고리형 컬럼
numeric_col_test = test.select_dtypes(include='int64').columns # 수치형 컬럼

In [126]:
#카테고리화 수치형으로 변환
ordinal_encoder = OrdinalEncoder()
test[categoric_col_test] = ordinal_encoder.fit_transform(test[categoric_col_test])

In [128]:
test

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
0,0.0,16800000,0.0,11.0,0.0,132000000,19.64,12,11.0,0,394692,146604.0,0.0,0.0
1,1.0,8400000,0.0,8.0,2.0,89971200,15.84,25,2.0,0,0,0.0,0.0,0.0
2,2.0,17280000,0.0,9.0,2.0,150000000,8.41,20,4.0,0,1786980,281820.0,0.0,0.0
3,3.0,14400000,0.0,8.0,0.0,66000000,13.72,30,4.0,1,669024,281724.0,0.0,0.0
4,4.0,27600000,0.0,8.0,2.0,55200000,30.50,12,4.0,0,1250052,614844.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64192,64192.0,30000000,0.0,6.0,0.0,78000000,22.08,27,2.0,2,1307532,763380.0,0.0,0.0
64193,64193.0,30000000,1.0,2.0,0.0,109200000,12.06,26,2.0,0,960612,1245252.0,0.0,0.0
64194,64194.0,6120000,0.0,2.0,2.0,39600000,28.80,33,2.0,0,131520,80880.0,0.0,0.0
64195,64195.0,11520000,0.0,2.0,0.0,66000000,25.44,41,2.0,1,1339536,601872.0,0.0,0.0


In [129]:
real = model.predict(test)

In [131]:
real

array([1, 0, 0, ..., 4, 2, 0])

In [136]:
real = np.where(real == 0, 'A',
                  np.where(real == 1, 'B',
                  np.where(real == 2, 'C',
                  np.where(real == 3, 'D',
                  np.where(real == 4, 'E', 'F')))))

In [139]:
submit['대출등급'] = real

In [140]:
submit.to_csv("submit.csv",index = False)

In [67]:

with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)